In [1]:
!pip install requests

In [ ]:
import numpy as np
import pandas as pd
import requests

In [2]:
# constants - base url and api key
BASE_URL = "https://sportsopendata.net/api/v1"
API_KEY = 'tbd'

# get data function
def get_football_data():
    # football endpoint
    endpoint = f"{BASE_URL}/football/games"
    
    # set up headers with API key
    headers = {
        'User-Agent': 'sharps',
        'apikey': API_KEY
    }

    try:
        # try request to the API
        response = requests.get(endpoint, headers=headers)
        response.raise_for_status()  # raise an error for bad responses
        
        # parse JSON
        data = response.json()
        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

In [3]:
# run function
football_data = get_american_football_data()

This is working


In [ ]:
print(football_data)